In [1]:
%cd ..

/home/den/dev/git/ozon-e-cup-2025


In [ ]:
import json

import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline

import src.preprocessing as preproc

In [ ]:
train_ids = np.load("data/train_ids.npy")
val_ids = np.load("data/val_ids.npy")

# Preprocess meta features

In [ ]:
meta_train = pd.read_csv("data/1__train_with_flags.csv")
meta_test = pd.read_csv("data/1__test_with_flags.csv")

In [ ]:
id2target = {int(id): int(target) for id, target in zip(meta_train["id"], meta_train["resolution"])}

json.dump(id2target, open("data/id2target.json", "w"))

In [ ]:
meta_train["SellerID"] = meta_train["SellerID"].astype("string")
meta_test["SellerID"] = meta_test["SellerID"].astype("string")

In [ ]:
preproc_pipe = Pipeline(
    steps=[
        ("num_preproc", preproc.NUMERIC_PREP_PIPELINE),
        ("cat_preproc", preproc.CATEGORIAL_PREP_PIPELINE),
    ]
)

In [ ]:
preproc_pipe = preproc_pipe.fit(meta_train[meta_train["id"].isin(train_ids)])

[Pipeline] ... (step 1 of 37) Processing rating_1_count, total=   0.1s
[Pipeline] ... (step 2 of 37) Processing rating_2_count, total=   0.1s
[Pipeline] ... (step 3 of 37) Processing rating_3_count, total=   0.1s
[Pipeline] ... (step 4 of 37) Processing rating_4_count, total=   0.1s
[Pipeline] ... (step 5 of 37) Processing rating_5_count, total=   0.1s
[Pipeline]  (step 6 of 37) Processing comments_published_count, total=   0.1s
[Pipeline]  (step 7 of 37) Processing photos_published_count, total=   0.1s
[Pipeline]  (step 8 of 37) Processing videos_published_count, total=   0.1s
[Pipeline] .. (step 9 of 37) Processing PriceDiscounted, total=   0.1s
[Pipeline] . (step 10 of 37) Processing item_time_alive, total=   0.1s
[Pipeline]  (step 11 of 37) Processing item_count_fake_returns7, total=   0.1s
[Pipeline]  (step 12 of 37) Processing item_count_fake_returns30, total=   0.1s
[Pipeline]  (step 13 of 37) Processing item_count_fake_returns90, total=   0.1s
[Pipeline]  (step 14 of 37) Proces

In [ ]:
meta_train_preproc = preproc_pipe.transform(meta_train[meta_train["id"].isin(train_ids)], is_train=True, is_val=False)
meta_val_preproc = preproc_pipe.transform(meta_train[meta_train["id"].isin(val_ids)], is_train=False, is_val=True)
meta_test_preproc = preproc_pipe.transform(meta_test, is_train=False, is_val=False)

/home/den/miniconda3/envs/ozon-e-cup/lib/python3.12/site-packages/sklearn/pipeline.py:61: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
/home/den/miniconda3/envs/ozon-e-cup/lib/python3.12/site-packages/sklearn/pipeline.py:61: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
/home/den/dev/git/ozon-e-cup-2025/src/preprocessing.py:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  series.loc[random_mask] = self._new_cat_name
/home/den/d

In [ ]:
meta_train_preproc.shape, meta_val_preproc.shape, meta_test_preproc.shape

((155979, 80), (41219, 80), (22760, 79))

In [ ]:
meta_train_preproc.drop(columns=["description", "name_rus", "ItemID", "resolution"], inplace=True)
meta_val_preproc.drop(columns=["description", "name_rus", "ItemID", "resolution"], inplace=True)
meta_test_preproc.drop(columns=["description", "name_rus", "ItemID"], inplace=True)

In [ ]:
meta_train_preproc.to_csv("data/meta_train.csv", index=False)
meta_val_preproc.to_csv("data/meta_val.csv", index=False)
meta_test_preproc.to_csv("data/meta_test.csv", index=False)

In [ ]:
meta_cols = meta_train_preproc.columns.tolist()

# Basic text features

In [ ]:
basic_text_train = pd.read_csv("data/X_train_text_features_basic.csv")
basic_text_test = pd.read_csv("data/X_test_text_features_basic.csv")

In [ ]:
basic_text_train.columns

Index(['id', 'cos_sim_name_rus_vs_description',
       'cos_sim_brand_name_vs_description', 'cos_sim_brand_name_vs_name_rus',
       'has_url', 'has_phone', 'has_messenger', 'has_sku', 'desc_len_chars',
       'desc_len_words', 'capslock_word_count', 'exclamation_count',
       'question_count', 'avg_word_length', 'has_price', 'upper_ratio',
       'has_emoji', 'emoji_count', 'avg_sentence_length',
       'punctuation_density', 'sentence_length_std', 'has_suspicious_words',
       'suspicious_word_count', 'has_brand', 'brand_count',
       'has_urgency_words', 'urgency_word_count', 'unique_word_ratio'],
      dtype='object')

In [ ]:
basic_text_cols = basic_text_train.columns.to_list()

In [ ]:
basic_text_train[basic_text_train["id"].isin(train_ids)].shape, basic_text_train[
    basic_text_train["id"].isin(val_ids)
].shape, basic_text_test.shape

((155979, 28), (41219, 28), (22760, 28))

In [ ]:
basic_text_train[basic_text_train["id"].isin(train_ids)].to_csv("data/basic_text_train.csv", index=False)
basic_text_train[basic_text_train["id"].isin(val_ids)].to_csv("data/basic_text_val.csv", index=False)
basic_text_test.to_csv("data/basic_text_test.csv", index=False)

In [ ]:
json.dump(
    {"meta": meta_cols, "basic_text": basic_text_cols},
    open("data/cols.json", "w"),
)